# Image Processing using PyTorch
## Author: Rohan Singh
In this notebook we will be using PyTorch for image processing. The reasoning behind using images in this notebook because as we saw in the first notebook on tensors, we saw that RGB images are a very easy way to visualize tensors.

## Libraries Used

In [15]:
# EDA libraries
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

# PyTorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)
from torchvision import models
from torchvision import transforms

# Image processing libraries
import imageio
import os

## Data Fetching

### Reading the image

In [6]:
# Change this
path_to_image = "/Users/rohansingh/Desktop/emoji pics/yastrzemski.jpeg"

img_arr = imageio.imread(path_to_image)
img_arr.shape

/var/folders/yb/n2ch33ws4yzf1nn686jzrjs40000gn/T/ipykernel_20935/1965604962.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(path_to_image)


(191, 264, 3)

In [7]:
type(img_arr)

numpy.ndarray

### Converting the image into a PyTorch Tensor

In [8]:
img = torch.from_numpy(img_arr)

In [9]:
img.shape

torch.Size([191, 264, 3])

In [10]:
out = img.permute(2, 0, 1)

In [12]:
out.shape

torch.Size([3, 191, 264])

## Tensor Time

In [22]:
batch_size = 7
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [19]:
batch.shape

torch.Size([7, 3, 256, 256])

In [29]:
data_dir = '/Users/rohansingh/Documents/img_tensor_batches/'

filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name)[-1] == '.png']

for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t

/var/folders/yb/n2ch33ws4yzf1nn686jzrjs40000gn/T/ipykernel_20935/3463533930.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


### Normalizing the tensor
In most use cases we want to normalize the tensor values so that each entry lies in the range [-1,1] or [0,1]. In the case of RGB images we can simply normalize them by dividing them with the max RGB value (that we convenitently know is 255).  

After getting them into the range of [0,1], we can then normalize the values such that the values have a mean of zero and unit standard deviation across each channel (R/G/B respectively).

In [30]:
batch = batch.float()
batch /= 255.0

In [33]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std